# Daily Mix

In [135]:
import pandas as pd
import numpy as np
import requests
import pymysql
import json
import os
import time
import shutil
import random
import cv2
from PIL import Image
import shutil
from datetime import datetime

In [2]:
def connect():    
    config = {
    "user":'admin',
    "password":"InDJ20190301!@",
    "host":"nas.indj.me",
    "database":"indj",
    "port": 3333
    }

    conn=pymysql.connect(**config)
    cursor = conn.cursor()
    
    return cursor

In [118]:
user_idx = 10

In [150]:
time = datetime.today().strftime("%Y_%m_%d %H:%M:%S")

cursor = connect()

sql = """
    SELECT tag_idx
    FROM user_tag 
    WHERE user_idx = """+ str(user_idx) +""" AND TYPE = 3
"""
cursor.execute(sql)

user_tag = cursor.fetchall()

tag = []
for i in range(len(user_tag)):
    tag.append(int(user_tag[i][0]))

sql = """
    SELECT tag_idx
    FROM user_tag 
    WHERE user_idx = """+ str(user_idx) +""" AND TYPE = 1
"""
cursor.execute(sql)

result = cursor.fetchall()

for i in range(len(result)):
    
    music_list = []
    music_title = []
    artist_name = []
    
    sql = """
        SELECT name
        FROM artist
        WHERE idx = """+ result[i][0] +"""
    """
    cursor.execute(sql)

    find_artist_name = cursor.fetchall()
    
    sql = """
            SELECT
            t.idx, t.name, COUNT(t.name) AS cnt
            FROM tag AS t
            JOIN artist_album_genre AS aag ON t.idx = aag.tag_idx
            JOIN artist_album_re AS aar ON aag.album_idx = aar.album_idx
            JOIN artist AS a ON aar.artist_idx = a.idx
            WHERE a.idx = """+ result[i][0] +"""
            GROUP BY t.name
            ORDER BY cnt desc
    """
    
    cursor.execute(sql)

    find_genre = cursor.fetchall()
    genre = find_genre[0][0]
    
    sql = """
        SELECT aam.idx, aam.`file`, t.idx, t.name
        FROM user_tag AS ut
        JOIN tag AS t ON ut.tag_idx = t.idx
        JOIN ai_channel_tag AS act ON t.idx = act.tag_idx
        JOIN ai_channel_music AS acm ON act.ai_channel_idx = acm.ai_channel_idx
        JOIN artist_album_music AS aam ON acm.music_idx = aam.idx
        JOIN artist_album_re AS aar ON aam.album_idx = aar.album_idx
        JOIN artist AS a ON aar.artist_idx = a.idx
        WHERE ut.user_idx = """+ str(user_idx) +""" AND a.idx = """+ result[i][0] +""" AND ut.`type` = 3 AND aam.`file` NOT LIKE '%inst.%' AND aam.`file` NOT LIKE '%_album%'
        GROUP BY aam.idx
    """
    cursor.execute(sql)

    result2 = cursor.fetchall()
    
    if result2 == ():
        sql = """
            SELECT aam.idx, aam.`file`
            FROM artist AS a
            JOIN artist_album_re AS aar ON a.idx = aar.artist_idx
            JOIN artist_album_music AS aam ON aar.album_idx = aam.album_idx
            WHERE a.idx = """+ result[i][0] +""" AND aam.`file` NOT LIKE '%inst.%' AND aam.`file` NOT LIKE '%_album%'
            GROUP BY aam.idx
        """
        cursor.execute(sql)
        result2 = cursor.fetchall()
   
    if len(result2) > 4:
        extract_list = random.sample(result2, k=4)
    else:
        extract_list = random.sample(result2, k=len(result2))
    
    for k in range(len(extract_list)):
        music_list.append(extract_list[k][0])
        music_title.append(extract_list[k][1])

    sql = """
        SELECT
        sml.sml_artist_idx, a.name
        FROM artist_sml AS sml
        JOIN artist AS a ON a.idx = sml.sml_artist_idx
        WHERE sml.artist_idx = """+ str(result[i][0]) +"""
    """
    cursor.execute(sql)

    result3 = cursor.fetchall()
    
    if result3 == ():
#         print('유사 x', result[i][0])
        continue

    for j in range(len(result3)):
        
        sql = """
        SELECT name
        FROM artist
        WHERE idx = """+ str(result3[j][0]) +"""
        """
        cursor.execute(sql)

        find_artist_name2 = cursor.fetchall()
        
        artist_name.append(find_artist_name2[0][0])

        sql = """
        SELECT aam.idx, aam.`file`, t.idx, t.name
        FROM user_tag AS ut
        JOIN tag AS t ON ut.tag_idx = t.idx
        JOIN ai_channel_tag AS act ON t.idx = act.tag_idx
        JOIN ai_channel_music AS acm ON act.ai_channel_idx = acm.ai_channel_idx
        JOIN artist_album_music AS aam ON acm.music_idx = aam.idx
        JOIN artist_album_re AS aar ON aam.album_idx = aar.album_idx
        JOIN artist AS a ON aar.artist_idx = a.idx
        WHERE ut.user_idx = 12 AND a.idx = """+ str(result3[j][0]) +""" AND ut.`type` = 3 AND aam.`file` NOT LIKE '%inst.%' AND aam.`file` NOT LIKE '%_album%'
        GROUP BY aam.idx
        """
        cursor.execute(sql)

        result4 = cursor.fetchall()

        if len(result4) == 0:
            sql = """
            SELECT aam.idx, aam.`file`
            FROM artist AS a
            JOIN artist_album_re AS aar ON a.idx = aar.artist_idx
            JOIN artist_album_music AS aam ON aar.album_idx = aam.album_idx
            WHERE a.idx = """+ str(result3[j][0]) +""" AND aam.`file` NOT LIKE '%inst.%' AND aam.`file` NOT LIKE '%_album%'
            GROUP BY aam.idx
            """
            cursor.execute(sql)
            result4 = cursor.fetchall()

        if len(result4) > 3:
            extract_list2 = random.sample(result4, k=3)
        else:
            extract_list2 = random.sample(result4, k=len(result4))

        for k in range(len(extract_list2)):
            music_list.append(extract_list2[k][0])
            music_title.append(extract_list2[k][1])    
    
    channel_artist = random.sample(artist_name, k=1)
    count = 0
    if len(music_list) > 17:
        random.shuffle(music_list)
        image_reshape(music_list, get_images)
        
        file_list = os.listdir('./back') 
        back_images = [file for file in file_list if file.endswith(".png")]  
        for b in range(len(back_images)):
            if genre == int(back_images[b].replace('.png','')):
                e = genre
                generate_image(e)
                break
            else:
                e = None
                continue
                
        if e != None:
            
            channel_name = find_artist_name[0][0] + ', ' + channel_artist[0] + '와(과) 유사한 아티스트 Daily Mix_'+time
            print(user_idx, channel_name, tag)
            print('music_list_len:',len(music_title))
            print(music_title)
            count += 1
        
    if count == 10:
        break
        


박효신, 더 네임(The Name)와(과) 유사한 아티스트 Daily Mix_2020_06_28 17:33:42 [470, 484, 298]
music_list_len: 27
['박효신-Beautiful Day (feat. 스컬)', '박효신-사랑...그 흔한 말', '박효신-Goodbye', '박효신-눈의 꽃', '포맨-해줄걸', '포맨-고백 (Produced By Vibe 윤민수)', '포맨-Baby Baby', '바이브-Promise U', '바이브-오래오래', '바이브-Where you are', '브라운 아이즈(Brown Eyes)-For You (돌아가줘)', '브라운 아이즈(Brown Eyes)-With Coffee...', '브라운 아이즈(Brown Eyes)-For You', '나얼(Na-Ul)-Spring Song', '나얼(Na-Ul)-Baby Funk', '나얼(Na-Ul)-그대 떠난 뒤', '더 네임(The Name)-그녀를 찾아주세요', '더 네임(The Name)-For You Live', '더 네임(The Name)-Used By You', '리치(Rich)-사랑해, 이 말 밖엔...', '원티드(Wanted)-너에게로 간다', '먼데이 키즈(Monday Kiz)-가을 안부', '먼데이 키즈(Monday Kiz), 펀치 (Punch)-Another Day', '먼데이 키즈(Monday Kiz)-모르시죠', '휘성(Realslow)-생각난다', '휘성(Realslow)-주르륵', '휘성(Realslow)-일년이면']
엠씨더맥스 (M.C the MAX), 규현 (KYUHYUN)와(과) 유사한 아티스트 Daily Mix_2020_06_28 17:33:42 [470, 484, 298]
music_list_len: 50
['엠씨더맥스 (M.C the MAX)-처음처럼', '엠씨더맥스 (M.C the MAX)-그대는 눈물겹다', '엠씨더맥스 (M.C the MAX)-잠시만 안녕 (Original)', '엠씨더맥스 (M.C the MAX)-사계